In [453]:
# import packages
import pandas as pd
import numpy as np
from pulp import *

import sqlite3

PATH: str = "C:\\Users\\User\\Desktop\\repos\\QPoland-Hackathon-24\\data\\"

# make a connection to the database (.db)
con = sqlite3.connect(PATH + "smplr_inst.db")

In [122]:
# read as a dataframe
flights = pd.read_sql_query("SELECT * FROM flights", con)
flights.leg_id = flights.leg_id.astype(str)
flights

,index,leg_id,flight_id,date,aircraft,from,to,start,finish,crew_group_id
0,0,70002970,2970,2006-01-07,A320#10,TLS,ORY,2006-01-07T06:40,2006-01-07T08:00,376980210
1,1,70002975,2975,2006-01-07,A320#10,ORY,TLS,2006-01-07T08:50,2006-01-07T10:00,376980210
2,2,70002982,2982,2006-01-07,A320#10,TLS,ORY,2006-01-07T10:40,2006-01-07T12:00,376980210
3,3,70002985,2985,2006-01-07,A320#10,ORY,TLS,2006-01-07T12:50,2006-01-07T14:00,376980210
4,4,70002992,2992,2006-01-07,A320#10,TLS,ORY,2006-01-07T14:40,2006-01-07T16:00,1878748607
5,5,70002999,2999,2006-01-07,A320#10,ORY,TLS,2006-01-07T16:50,2006-01-07T18:00,1878748607
6,6,70003008,3008,2006-01-07,A320#10,TLS,ORY,2006-01-07T18:40,2006-01-07T20:00,1827753950
7,7,70004563,4563,2006-01-07,A321#4,CDG,NCE,2006-01-07T06:20,2006-01-07T07:55,623057530
8,8,70004564,4564,2006-01-07,A321#4,NCE,CDG,2006-01-07T09:05,2006-01-07T10:40,623057530
9,9,70004567,4567,2006-01-07,A321#4,CDG,NCE,2006-01-07T11:35,2006-01-07T13:10,623057530


In [123]:
routes = pd.read_sql_query("SELECT * FROM routes", con)
routes

,index,from,to,type,distance
0,0,CDG,NCE,D,95
1,1,CDG,ORY,D,35
2,2,CDG,TLS,D,85
3,3,NCE,CDG,D,95
4,4,NCE,ORY,D,85
5,5,NCE,TLS,D,85
6,6,ORY,CDG,D,35
7,7,ORY,NCE,D,85
8,8,ORY,TLS,D,80
9,9,TLS,CDG,D,90


In [173]:
aircraft = pd.read_sql_query("SELECT * FROM aircraft", con)
aircraft

,index,registration,model,family,position,owned
0,0,A321#4,A321,Airbus,CDG,1
1,1,A320#10,A320,Airbus,TLS,1


In [125]:
# create time buckets
# Using discrete time-buckets
min_bucket: int = 5  # min

# Create time-series between arrival of first plane and departure of last
t = pd.Series(
    pd.date_range(
        start=flights["start"].min(),
        end=flights["finish"].max(),
        freq=pd.offsets.Minute(min_bucket),
    ),
)

A = pd.DataFrame(np.zeros((len(t), len(flights.leg_id))), index=t)
A.columns = flights.leg_id
A

leg_id,70002970,70002975,70002982,70002985,70002992,70002999,70003008,70004563,70004564,70004567,70004568,70004571,70004572
2006-01-07 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-01-07 20:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Integer Linear Programing approach

### Initialise model

In [323]:
lp = LpProblem("Flight_Scheduler", LpMinimize)  # minimize cost

### Define Variables where $time_i$ is allocated to $leg_j$

$x[time,leg] \in (0,inf)$ and $y[time,leg] \in \{0, 1\}$, where $x$ is variable amd $y$ is a non-zero indicator

*Note: x varaibles were to be use to store all the information about additional costs of the choosen timestamp, while y variables were to be used as the indicators for the constrains.*

In [324]:
x = {}
for leg in A.columns:
    for time in A.index:
        x[time, leg] = LpVariable(f"x_{time}_{leg}", lowBound=0, cat=LpInteger)

y = {}
for leg in A.columns:
    for time in A.index:
        y[time, leg] = LpVariable(f"y_{time}_{leg}", cat=LpBinary)

### Set the objective function as sum of the X matrix (for now)
$min(\sum_{i=0}^{|time|}\sum_{j=0}^{|leg|}{x[i,j]})$

In [325]:
lp += A.sum().sum(), "lost"

### Constrains

#### i) Each leg must be assigned to only one time stamp

In [326]:
for leg in A.columns:
    lp += lpSum(y[time, leg] for time in A.index) == 1, f"One_nonzero_in_column_{leg}"

# dependence bewtwween x an y
M = 10_000  # constrain value for x[i, j]
for leg in A.columns:
    for time in A.index:
        lp += x[time, leg] <= M * y[time, leg], f"Link_x_y_{time}_{leg}"

### ii) Each time stamp must have no more assigned then the number of available planes

In [327]:
for t in A.index:
    lp += (
        lpSum(y[time, leg] for leg in A.columns)
        <= aircraft[aircraft.owned == 1].shape[0]
    )

### Pending constrains
- Aircraft location restricitions (Kirchhoff's Laws)
- Adding maintainance of the airplanes on the given flights (legs) $x[leg, time]$
- Adding the airport clousers on the given flights (legs) $x[leg, time]$
- Reduction in the availability of staff and equipment
- Distances between airports

### Solving the problem

In [328]:
# Solve
lp.solve()

# Report
print("Status: ", LpStatus[lp.status])
print("Minimised Cost: ", value(lp.objective))

Status:  Optimal
Minimised Cost:  None


### Preparing the updated schedule

In [413]:
results = A.copy()
for leg in A.columns:
    for time in A.index:
        results.loc[time, leg] = value(x[time, leg])
results

leg_id,70002970,70002975,70002982,70002985,70002992,70002999,70003008,70004563,70004564,70004567,70004568,70004571,70004572
2006-01-07 06:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 06:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-01-07 20:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-07 21:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [414]:
# addition of a random solution to prepare the form of the code
results = results.applymap(lambda x: np.random.randint(0, 100))
results = results.where(results.eq(results[results > 0].min(), axis=1), 0)
results = results[results > 0].dropna(how="all")
results

C:\Users\User\AppData\Local\Temp\ipykernel_10712\3006735880.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results = results.applymap(lambda x: np.random.randint(0, 100))


leg_id,70002970,70002975,70002982,70002985,70002992,70002999,70003008,70004563,70004564,70004567,70004568,70004571,70004572
2006-01-07 07:40:00,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 07:55:00,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 08:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2006-01-07 08:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2006-01-07 09:05:00,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 10:25:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 11:00:00,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 11:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2006-01-07 11:50:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-07 13:25:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [451]:
# find values
start = []
leg_id = []
for leg in results.columns:
    start.append(
        pd.to_datetime(
            results[leg]
            .dropna()
            .drop_duplicates()
            .index.strftime("%Y-%m-%d %H:%M:%S")
            .tolist()[0]
        )
    )
    leg_id.append(leg)

new_schedule = pd.DataFrame({"leg_id": leg_id, "start": start})
new_schedule["date"] = new_schedule["start"].dt.strftime("%Y-%m-%d")
new_schedule = pd.merge(
    new_schedule,
    flights.loc[:, ["leg_id", "flight_id", "aircraft", "from", "to", "crew_group_id"]],
    how="left",
    on="leg_id",
)

new_schedule = (
    pd.merge(
        new_schedule,
        routes.loc[:, ["from", "to", "distance"]],
        how="left",
    )
    .assign(finish=lambda x: x.start + pd.to_timedelta(x.distance, unit="minutes"))
    .drop("distance", axis=1)
    .loc[
        :,
        [
            "leg_id",
            "flight_id",
            "date",
            "aircraft",
            "from",
            "to",
            "start",
            "finish",
            "crew_group_id",
        ],
    ]
)

new_schedule

,leg_id,flight_id,date,aircraft,from,to,start,finish,crew_group_id
0,70002970,2970,2006-01-07,A320#10,TLS,ORY,2006-01-07 10:25:00,2006-01-07 11:45:00,376980210
1,70002975,2975,2006-01-07,A320#10,ORY,TLS,2006-01-07 07:40:00,2006-01-07 09:00:00,376980210
2,70002982,2982,2006-01-07,A320#10,TLS,ORY,2006-01-07 07:55:00,2006-01-07 09:15:00,376980210
3,70002985,2985,2006-01-07,A320#10,ORY,TLS,2006-01-07 17:40:00,2006-01-07 19:00:00,376980210
4,70002992,2992,2006-01-07,A320#10,TLS,ORY,2006-01-07 19:20:00,2006-01-07 20:40:00,1878748607
5,70002999,2999,2006-01-07,A320#10,ORY,TLS,2006-01-07 09:05:00,2006-01-07 10:25:00,1878748607
6,70003008,3008,2006-01-07,A320#10,TLS,ORY,2006-01-07 11:50:00,2006-01-07 13:10:00,1827753950
7,70004563,4563,2006-01-07,A321#4,CDG,NCE,2006-01-07 08:20:00,2006-01-07 09:55:00,623057530
8,70004564,4564,2006-01-07,A321#4,NCE,CDG,2006-01-07 08:55:00,2006-01-07 10:30:00,623057530
9,70004567,4567,2006-01-07,A321#4,CDG,NCE,2006-01-07 17:15:00,2006-01-07 18:50:00,623057530


In [452]:
# forward into the database
new_schedule.to_sql(name="new_schedule", con=con)
con.commit()